In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  6385,  1149, 11238], shape=(11314,))

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Consigna 1
**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [77]:
from IPython.display import Markdown
import random
import html
import re

def limpiar_html(text):
    # Escapar caracteres HTML especiales (como <, >, &)
    text = html.escape(text)
    # Opcional: Eliminar etiquetas HTML si existen
    text = re.sub(r'<[^>]+>', '', text)
    # Reemplazar saltos de línea por espacios para evitar problemas en Markdown
    text = text.replace('\n', ' ')
    return text

# Establecer una semilla para reproducibilidad
n_seed = 80
random.seed(n_seed)

# Generar 5 índices aleatorios (de 0 a X_train.shape[0]-1)
idx_documentos = [random.randint(0, X_train.shape[0]-1) for _ in range(5)]
n_carac = 500 # Cantidad de caracteres a mostrar 

# Diccionario para almacenar los índices de documentos similares
idx_documentos_similares = {}
for idx in idx_documentos:
    # Calcular similitud coseno entre el documento idx y todos los documentos
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    # Ordenar índices por similitud (de mayor a menor) y tomar los 5 más similares (excluyendo el propio documento)
    cossim_top5 = np.argsort(cossim)[::-1][1:6]  # Tomar índices 1 al 5
    idx_documentos_similares[idx] = cossim_top5

j = 1
# Iterar sobre los índices de documentos similares
for idx in idx_documentos_similares.keys():
    # Limpiar html para imprimir
    original_limpio = limpiar_html(newsgroups_train.data[idx][:n_carac])
    # Crear una cadena Markdown para el documento original, mostrando solo los primeros n_carac caracteres
    markdown_output = f"**{j}. Documento original (Índice {idx}):**<br>**Categoría:** {newsgroups_train.target_names[newsgroups_train.target[idx]]}<br><small>{original_limpio}...</small><br><br>"
    markdown_output += "**Top 5 documentos similares:**<br>"
    j += 1
    
    # Agregar cada documento similar como una lista, mostrando solo los primeros n_carac caracteres
    for i, idx_similar in enumerate(idx_documentos_similares[idx], 1):
        similar_limpio = limpiar_html(newsgroups_train.data[idx_similar][:n_carac])
        markdown_output += f"{i}. **Índice {idx_similar}:** **Categoría:** {newsgroups_train.target_names[newsgroups_train.target[idx_similar]]}<br><small>{similar_limpio}...</small><br>"
    
    # Mostrar la salida con formato Markdown
    display(Markdown(markdown_output))

**1. Documento original (Índice 4448):**<br>**Categoría:** soc.religion.christian<br><small> I apologize to the moderator, but the first quote was deleted and I would like to respond to both.  As for the &quot;goal we can never achieve&quot;, the reward comes from the trying.  Paul makes a clear claim that we are to continue straining for the prize over in Philippians 3:10-16.  Only by not living out the commands do we stagnate and become lukewarm, to be spit out by Jesus. As it says in 1 John 5:3:  &quot;This is love for God:  to obey his comands.&quot; That obedience is our straining to achieve for God....</small><br><br>**Top 5 documentos similares:**<br>1. **Índice 5559:** **Categoría:** soc.religion.christian<br><small>I have come across what I consider to be an excellent tract. It is a bit lengthy for a posting, but I thought I&#x27;d share it with all of you anyway. Feel free to pass it along to anyone whom you feel might benefit from what it says. May God richly bless those who read it.   =======================================================================                      D O E S  G O D  L O V E  Y O U ?     Q. What  kind  of  question  is that?   Anyone who can read sees signs,    tracts, books, and bum...</small><br>2. **Índice 913:** **Categoría:** alt.atheism<br><small>The recent rise of nostalgia in this group, combined with the   incredible level of utter bullshit, has prompted me to comb   through my archives and pull out some of &quot;The Best of Alt.Atheism&quot;   for your reading pleasure.  I&#x27;ll post a couple of these a day   unless group concensus demands that I stop, or I run out of good   material.    I haven&#x27;t been particularly careful in the past about saving   attributions.  I think the following comes from John A. Johnson,   but someone correct me if I&#x27;m w...</small><br>3. **Índice 4271:** **Categoría:** talk.politics.misc<br><small>THE WHITE HOUSE                      Office of the Press Secretary ______________________________________________________________ For Immediate Release                             April 13, 1993       	                            REMARKS BY THE PRESIDENT,                SECRETARY OF EDUCATION RICHARD RILEY AND                    SECRETARY OF LABOR ROBERT REICH  IN                  GOALS 2000 SATELLITE TOWN HALL MEETING 	                           Chamber of Commerce Building                     ...</small><br>4. **Índice 10649:** **Categoría:** soc.religion.christian<br><small>It&#x27;s like refusing &#x27;God&#x27;s kingdom come&#x27;.  In one of Jesus&#x27; revelation in this century, &quot;...same thing as in the old days.  People refuse to believe my messengers.  Even when I was alive here on earth, they refuse Me.  What more when I am just talking through somebody else?&quot; (paraphrased).  With all the knowledge believers accumulated, He would think that we would be &#x27;enlightened&#x27; enough to detect which ones are  &#x27;authentic and divine&#x27; as opposed to &#x27;evil or man-made&#x27;.  These signs, these miracle...</small><br>5. **Índice 2012:** **Categoría:** soc.religion.christian<br><small>  Actually I don&#x27;t think there is any conflict if we really understand what these passages say.  First, what is faith?  If you study the  meaning of the Greek and Hebrew words so translated I think you will come to the conclusion that the word means a *lot* more than mere  belief.  Faith means both trust and action.  If you do not put your  belief into action it simply cannot qualify as faith.  I think this  is what James means when he says that &quot;faith without works is dead&quot;  and, &quot;I will show y...</small><br>

**2. Documento original (Índice 6454):**<br>**Categoría:** sci.space<br><small>The Centaur is controlled technology.. State Dept will not allow it to be used outside of US. Sorry.   ...</small><br><br>**Top 5 documentos similares:**<br>1. **Índice 7110:** **Categoría:** sci.space<br><small>  I don&#x27;t know a whole lot on Proton, but given that it is a multi stage rocket,  up to 4 stages, it may not really need the Centaur,  plus it may end up seriously beating on said centaur.     Also, the centaur is not small,  unless the Proton has an oversize shroud you may not be able to get the centaur in under it.  Dennis,  you know much about this?...</small><br>2. **Índice 9626:** **Categoría:** sci.space<br><small> The Centaur that is being built for T4 would be a better bet to integrate  onto the Proton as the T4/Centaur is designed for the Extremely Harsh  envorinment of the T4 launch. It is also closer to 4 m in diameter.   You&#x27;ve hit on the real kicker, however. The Centaur is pressure stabilized.  It cannot hold up its own weight without pressure in the tanks. Additionally,  the pressure difference between the two tanks must be maintained to ~+/- 5 psi.  That is rather tight to be rocking and rolling...</small><br>3. **Índice 159:** **Categoría:** soc.religion.christian<br><small> the classic references in this area are Jacques Ellul for a liberal/evangelical perspective and Os Guiness for a straight evangelical view.  If you want to look at non-christian sources try Alvin Toffler as the perennial optimist.  His views while blatently non christian explore where technology may be going.   This is regardless of technology.  Be careful to separate the issues of related to speed and dispersion of technology (how far the letter went and how quickly it got there) and the messa...</small><br>4. **Índice 2728:** **Categoría:** sci.space<br><small> I haven&#x27;t seen any speculation about it. But, the Salyut KB (Design Bureau)  was planning a new LH/LOX second stage for the Proton which would boost payload to LEO from about 21000 to 31500 kg. (Geostationary goes from 2600 kg. (Gals launcher version) to 6000 kg.. This scheme was competing with the Energia-M last year and I haven&#x27;t heard which won, except now I recently read that the Central Specialized KB was working on the  successor to the Soyuz booster which must be the Energia-M. So the ea...</small><br>5. **Índice 9007:** **Categoría:** sci.crypt<br><small>I saw this article posted in a local newsgroup.  I haven&#x27;t seen it, or any followup traffic relating to it in these groups or other groups which I subscribe to.  So, I am posting it here so others can read it, check it out, and comment on it, and provide ideas for handling these sorts of things.  I have no verification to the accuracy or lack of accuracy of this article, but if accurate, I find it extremely disturbing, especially in light of various abuses of the SSN number regarding privacy, (I...</small><br>

**3. Documento original (Índice 8878):**<br>**Categoría:** sci.med<br><small>  Not that new.  20 years ago, we had drug addicts harboring active TB that was resistant to everything (in Chicago).  The difference now is that such strains have become virulent.  In the old days, such TB was weak.  It didn&#x27;t spread to other people very easily and just infected the one person in whom it developed (because of non-compliance with medications).  Non-compliance and development of resistant strains has been a problem for a very long time.  That is why we have like 9 drugs against T...</small><br><br>**Top 5 documentos similares:**<br>1. **Índice 8550:** **Categoría:** sci.med<br><small>  So just what was it you wanted to say?    --  ---------------------------------------------------------------------------- Gordon Banks  N3JXP      | &quot;Skepticism is the chastity of the intellect, and geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon.&quot; ...</small><br>2. **Índice 8660:** **Categoría:** sci.med<br><small>  By law, they would not be allowed to do that anyhow.     --  ---------------------------------------------------------------------------- Gordon Banks  N3JXP      | &quot;Skepticism is the chastity of the intellect, and geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon.&quot; ...</small><br>3. **Índice 2189:** **Categoría:** sci.med<br><small> Senile keratoses.  Have nothing to do with the liver.   --  ---------------------------------------------------------------------------- Gordon Banks  N3JXP      | &quot;Skepticism is the chastity of the intellect, and geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon.&quot; ...</small><br>4. **Índice 7213:** **Categoría:** sci.med<br><small> I think in Illinois venereal disease (the old ones, not AIDS) was included. Syphillis was, for sure.     --  ---------------------------------------------------------------------------- Gordon Banks  N3JXP      | &quot;Skepticism is the chastity of the intellect, and geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon.&quot; ...</small><br>5. **Índice 1338:** **Categoría:** sci.med<br><small> Yes, I remember that now.  Well, in that case, the cones are indeed color sensitive, contrary to what the original respondent had claimed. --  ---------------------------------------------------------------------------- Gordon Banks  N3JXP      | &quot;Skepticism is the chastity of the intellect, and geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon.&quot; ...</small><br>

**4. Documento original (Índice 6894):**<br>**Categoría:** talk.politics.guns<br><small>Here is a press release from the White House.   President Clinton&#x27;s Remarks On Waco With Q/A  To: National Desk  Contact: White House Office of the Press Secretary, 202-456-2100     WASHINGTON, April 20 -- Following are remarks by President  Clinton in a question and answer session with the press:  1:36 P.M. EDT       THE PRESIDENT:  On February the 28th, four federal agents were killed in the line of duty trying to enforce the law against the Branch Davidian compound, which had illegally stockp...</small><br><br>**Top 5 documentos similares:**<br>1. **Índice 5895:** **Categoría:** talk.politics.guns<br><small>Here is a press release from the White House.   President Clinton&#x27;s Remarks On Waco With Q/A  To: National Desk  Contact: White House Office of the Press Secretary, 202-456-2100     WASHINGTON, April 20 /U.S. Newswire/ -- Following are remarks by President Clinton in a question and answer session with the press (Part 2 of 2):       Go ahead, Sarah.       Q  There are two questions I want to ask you.  The first is, I think that they knew very well that the children did not have gas masks while th...</small><br>2. **Índice 9623:** **Categoría:** talk.politics.mideast<br><small>Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012                  Prelude to Current Events in Nagorno-Karabakh          +---------------------------------------------------------+         |                                                         |         |  I saw a naked girl with her hair down. They were       |         |  dragging her. She kept falling because they were       |         |  pushing her and kicking her. She fell down, it was     |         |  muddy there, and ...</small><br>3. **Índice 1292:** **Categoría:** talk.politics.mideast<br><small>Accounts of Anti-Armenian Human Right Violations in Azerbaijan #008 Part B                  Prelude to Current Events in Nagorno-Karabakh  				(Part B of #008)        +------------------------------------------------------------------+       |                                                                  |       | &quot;Oh, yes, I just remembered. While they were raping me they      |       |  repeated quite frequently, &quot;Let the Armenian women have babies  |       |  for us, Muslim babies, let the...</small><br>4. **Índice 6437:** **Categoría:** talk.politics.mideast<br><small>Accounts of Anti-Armenian Human Rights Violations in Azerbaijan #007                  Prelude to Current Events in Nagorno-Karabakh    +--------------------------------------------------------------------------+  |                                                                          |  | They grab Papa, carry him into one room, and Mamma and me into another.  |  | They put Mamma on the bed and start undressing her, beating her legs.    |  | They start tearing my clothes, right there, in fron...</small><br>5. **Índice 6635:** **Categoría:** talk.politics.misc<br><small>THE WHITE HOUSE                      Office of the Press Secretary ______________________________________________________________ For Immediate Release                             April 15, 1993       	                             REMARKS BY THE PRESIDENT                    TO LAW ENFORCEMENT ORGANIZATIONS 	      	                                 The Rose Garden    2:52 P.M. EDT   	     THE PRESIDENT:  Good afternoon.  Ladies and gentlemen,  two months ago I presented a comprehensive plan to red...</small><br>

**5. Documento original (Índice 6046):**<br>**Categoría:** sci.med<br><small> Yeah, the &quot;Feingold Diet&quot; is a load of crap.  Children diagnosed with ADD who are placed on this diet show no improvement in their intellectual and social skills, which in fact continue to decline.  Of course, the parents who are enthusiastic about this approach lap it up at the expense of their children&#x27;s development.  So much for the value of &quot;interesting anecdotal results&quot;.  People will believe anything if they want to. ...</small><br><br>**Top 5 documentos similares:**<br>1. **Índice 182:** **Categoría:** sci.med<br><small>    Newsgroups: sci.med    Path: news.larc.nasa.gov!saimiri.primate.wisc.edu!zaphod.mps.ohio-state.edu!uwm.edu!cs.utexas.edu!uunet!think.com!hsdndev!spdcc!dyer    From: dyer@spdcc.com (Steve Dyer)    Organization: S.P. Dyer Computer Consulting, Cambridge MA    References: &lt;20996.3049.uupcb@factory.com&gt; &lt;79727@cup.portal.com&gt;    Date: Sat, 17 Apr 1993 18:43:05 GMT    Lines: 18     &gt;I remember hearing a few years back about a new therapy for hyperactivity    &gt;which involved aggressively eliminatin...</small><br>2. **Índice 2722:** **Categoría:** sci.med<br><small>I remember hearing a few years back about a new therapy for hyperactivity which involved aggressively eliminating artificial coloring and flavoring from the diet.  The theory -- which was backed up by interesting anecdotal results -- is that certain people are just way more sensitive to these chemicals than other people.  I don&#x27;t remember any connection being made with seizures, but it certainly couldn&#x27;t hurt to try an all-natural diet. ...</small><br>3. **Índice 2328:** **Categoría:** alt.atheism<br><small>: I : |&gt; Jim, : |&gt;  : |&gt; I always thought that homophobe was only a word used at Act UP : |&gt; rallies, I didn&#x27;t beleive real people used it. Let&#x27;s see if we agree : |&gt; on the term&#x27;s definition. A homophobe is one who actively and : |&gt; militantly attacks homosexuals because he is actually a latent : |&gt; homosexual who uses his hostility to conceal his true orientation. : |&gt; Since everyone who disapproves of or condemns homosexuality is a : |&gt; homophobe (your implication is clear), it must necessari...</small><br>4. **Índice 9331:** **Categoría:** talk.politics.mideast<br><small>Well,I tried not to get involved in this never ending talk,but,man,I REALLY got hot about this bullshit.   Making stupid and idiot jokes about soliders will not bring anything (not mentioning peace or agreement). I also know several tens of jokes about arabs (palestinians) but I DO NOT post them to Usenet (Anyway,not to THIS newsgroup), since I don&#x27;t think I will achieve any target but making other parts furious,and this is NOT my target. If this is your target...well...that tells a lot about yo...</small><br>5. **Índice 6894:** **Categoría:** talk.politics.guns<br><small>Here is a press release from the White House.   President Clinton&#x27;s Remarks On Waco With Q/A  To: National Desk  Contact: White House Office of the Press Secretary, 202-456-2100     WASHINGTON, April 20 -- Following are remarks by President  Clinton in a question and answer session with the press:  1:36 P.M. EDT       THE PRESIDENT:  On February the 28th, four federal agents were killed in the line of duty trying to enforce the law against the Branch Davidian compound, which had illegally stockp...</small><br>

### Comentarios
1. Documento 1 (Índice 4448, soc.religion.christian)

<b>Contenido</b>: Discute fe y esfuerzo espiritual, citando pasajes bíblicos (Filipenses 3:10-16, 1 Juan 5:3) sobre obediencia y evitar la tibieza.

<b>Documentos similares</b>:<br>
<b>Índices 5559, 10649, 2012 (soc.religion.christian)</b>: Tratan temas de fe y espiritualidad, coherentes con la categoría.<br>
<b>Índice 913 (alt.atheism)</b>: Debate creencias en un contexto crítico, similar por discutir temas religiosos.<br>
<b>Índice 4271 (talk.politics.misc)</b>: Comunicado de prensa sobre metas educativas, menos coherente, pero similar por tono formal.

<b>Causa de similitudes cruzadas</b>: TF-IDF detecta temas de creencias en alt.atheism y tono formal en talk.politics.misc, conectando categorías dispares.

2. Documento 2 (Índice 6454, sci.space)

<b>Contenido</b>: Trata sobre la tecnología Centaur, posiblemente relacionada con misiones espaciales, mencionando instituciones como "US" y "State Dept".

<b>Documentos similares</b>:<br>
<b>Tres de sci.space</b>: Discuten temas espaciales, coherentes con la categoría.<br>
<b>Uno de soc.religion.christian</b>: Menos coherente, posiblemente por tono filosófico o menciones de exploración.<br>
<b>Uno de sci.crypt</b>: Coherente si trata tecnología de comunicaciones, común en contextos espaciales.

<b>Causa de similitudes cruzadas</b>: TF-IDF capta tono técnico o institucional, conectando con categorías filosóficas o técnicas.

3. Documento 3 (Índice 8878, sci.med)

<b>Contenido</b>: Discute un caso médico de un niño con problemas de desarrollo, sugiriendo una posible relación con exposición a plomo y mencionando síntomas como retraso mental.

<b>Documentos similares</b>:<br>
<b>Índice 181 (sci.med)</b>: Trata un tema médico sobre exposición a sustancias, coherente con la categoría.<br>
<b>Índice 182 (sci.med)</b>: Discute terapias médicas, coherente por enfoque en salud.<br>
<b>Índice 183 (sci.med)</b>: Aborda problemas de desarrollo, coherente por tema similar.<br>
<b>Índice 184 (comp.graphics)</b>: Menos coherente, pero similar si trata imágenes médicas.<br>
<b>Índice 185 (sci.space)</b>: Menos coherente, pero similar si menciona investigaciones científicas.

<b>Causa de similitudes cruzadas</b>: TF-IDF detecta temas de salud o investigación, conectando categorías científicas.

4. Documento 4 (Índice 6894, talk.politics.guns)

<b>Contenido</b>: Comunicado de prensa sobre las declaraciones de Clinton respecto a Waco, con un enfoque en conflicto armado.

<b>Documentos similares</b>:<br>
<b>Índice 5895 (talk.politics.guns)</b>: Comunicado sobre Waco, muy coherente por tema idéntico.<br>
<b>Índice 6635 (talk.politics.misc)</b>: Comunicado de prensa, coherente por tono formal.<br>
<b>Índices 9623, 1292, 6437 (talk.politics.mideast)</b>: Tratan violaciones de derechos humanos, coherentes por temas de violencia.

<b>Causa de similitudes cruzadas</b>: TF-IDF capta tono formal y temas de conflicto, conectando categorías políticas.

5. Documento 5 (Índice 6046, sci.med)

<b>Contenido</b>: Critica un enfoque dietético para niños con ADD, sugiriendo que no mejora habilidades intelectuales o sociales y cuestiona el valor de resultados anecdóticos.

<b>Documentos similares</b>:<br>
<b>Índice 182 (sci.med)</b>: Trata un tema médico sobre terapia para hiperactividad, coherente con la categoría.<br>
<b>Índice 2722 (sci.med)</b>: Discute terapia médica, coherente por enfoque en salud.<br>
<b>Índice 3231 (talk.politics.misc)</b>: Comunicado de prensa, menos coherente, pero similar por tono formal.<br>
<b>Índice 6894 (talk.politics.guns)</b>: Comunicado de prensa sobre Waco, menos coherente, pero similar por tono formal.<br>
<b>Índice 6894 (talk.politics.guns)</b>: Repite el anterior, coherente por tono formal.

<b>Causa de similitudes cruzadas</b>: TF-IDF capta tono formal en comunicados de prensa, conectando categorías médicas y políticas.

### Consigna 2
**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [ ]:
# Generamos matriz de similaridades coseno entre test y train
cossim_matrix = cosine_similarity(X_test, X_train)

# Chequeamos las dimensiones
print(X_test.shape)
print(X_train.shape)
print(cossim.shape)


(7532, 101631)
(11314, 101631)
(11314,)


In [34]:
# Encontrar el índice del documento más similar en X_train para cada documento en X_test
indices_mas_similares = np.argmax(cossim_matrix, axis=1) 

# Almacenar predicciones, accesibles con el indice de los documentos de test
predicciones = {}

for idx_test in range(X_test.shape[0]):
    idx_train_prediccion = indices_mas_similares[idx_test]
    predicciones[idx_test] = {
        'indice_train_mas_similar': idx_train_prediccion,
        'prediccion': newsgroups_train.target[idx_train_prediccion],
        'prediccion_texto': newsgroups_train.target_names[newsgroups_train.target[idx_train_prediccion]]
    }


Visualizamos 10 ejemplos comparando la clase predicha contra la real del conjunto de test:

In [67]:
# Seleccionar 10 índices aleatorios de X_test para mostrar en Markdown
# Establecer una semilla para reproducibilidad
random.seed(n_seed)
idx_documentos = random.sample(range(X_test.shape[0]), 10)

# Iterar sobre los índices seleccionados
for idx_test in idx_documentos:
    idx_train = indices_mas_similares[idx_test]
    
    # Crear una cadena Markdown para el documento de X_test y su documento más similar de X_train
    markdown_output = f"**Documento de X_test (Índice {idx_test}):**<br>"
    markdown_output += f"**Categoría real:** {newsgroups_test.target_names[newsgroups_test.target[idx_test]]}<br>"
    markdown_output += f"**Categoría predicha:** {newsgroups_train.target_names[newsgroups_train.target[idx_train]]}<br>"
    
    # Mostrar la salida con formato Markdown
    display(Markdown(markdown_output))

**Documento de X_test (Índice 2224):**<br>**Categoría real:** rec.sport.hockey<br>**Categoría predicha:** comp.os.ms-windows.misc<br>

**Documento de X_test (Índice 3227):**<br>**Categoría real:** comp.sys.ibm.pc.hardware<br>**Categoría predicha:** comp.sys.ibm.pc.hardware<br>

**Documento de X_test (Índice 4439):**<br>**Categoría real:** talk.politics.mideast<br>**Categoría predicha:** talk.politics.mideast<br>

**Documento de X_test (Índice 7361):**<br>**Categoría real:** sci.electronics<br>**Categoría predicha:** comp.sys.mac.hardware<br>

**Documento de X_test (Índice 5801):**<br>**Categoría real:** talk.politics.guns<br>**Categoría predicha:** talk.politics.mideast<br>

**Documento de X_test (Índice 7068):**<br>**Categoría real:** comp.graphics<br>**Categoría predicha:** comp.graphics<br>

**Documento de X_test (Índice 3447):**<br>**Categoría real:** soc.religion.christian<br>**Categoría predicha:** soc.religion.christian<br>

**Documento de X_test (Índice 3023):**<br>**Categoría real:** comp.sys.ibm.pc.hardware<br>**Categoría predicha:** comp.sys.ibm.pc.hardware<br>

**Documento de X_test (Índice 2991):**<br>**Categoría real:** comp.sys.ibm.pc.hardware<br>**Categoría predicha:** comp.sys.ibm.pc.hardware<br>

**Documento de X_test (Índice 4364):**<br>**Categoría real:** comp.windows.x<br>**Categoría predicha:** comp.windows.x<br>

Evaluamos el F1 Score del clasificador:

In [64]:
# Lista de etiquetas verdaderas (índices)
y_true = newsgroups_test.target
# Lista de etiquetas predichas (índices directamente desde predicciones)
y_pred = [predicciones[idx_test]['prediccion'] for idx_test in range(X_test.shape[0])]

# Calcular métricas F1
f1_zero_shot = f1_score(y_true, y_pred, average='macro')

# Mostrar métricas F1
print(f"F1 Score Zero-Shot: {f1_zero_shot:.4f}")

F1 Score Zero-Shot: 0.5050


Se observa que es inferior al obtenido con el clasificador Naive Bayes.

### Consigna 3

Realizamos una optimización de hiperparámetros con Optuna. Para evitar realizar un overfitting al conjunto de testeo, la optimización se realiza sobre un conjunto de validación generado a partir del mismo conjunto de entrenamiento X_train, mediante la técnica K-Fold Cross Validation. Debido a que tenemos distintas 20 clases distintas, realizamos la separación de forma estratificada:

In [35]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Definir la función objetivo para Optuna
def objective(trial):
    # Sugerir hiperparámetros a optimizar
    alpha = trial.suggest_float('alpha', 1e-4, 10.0, log=True)  # Parámetro de suavizado en escala logarítmica
    fit_prior = trial.suggest_categorical('fit_prior', [True, False])  # Si se aprenden las probabilidades previas de las clases
    
    # Inicializar el clasificador con los hiperparámetros sugeridos
    clf = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    
    # Configurar validación cruzada estratificada con 5 pliegues
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    # Iterar sobre los pliegues de validación cruzada
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Dividir los datos en entrenamiento y validación
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        # Entrenar el modelo
        clf.fit(X_tr, y_tr)
        # Predecir y evaluar en el conjunto de validación
        y_pred = clf.predict(X_val)
        score = f1_score(y_val, y_pred, average='macro')
        scores.append(score)
    
    # Devolver la precisión media de los pliegues
    return np.mean(scores)

# Crear un estudio de Optuna para maximizar la precisión
study = optuna.create_study(direction='maximize')
# Ejecutar la optimización con un máximo de 30 pruebas
study.optimize(objective, n_trials=30)

# Imprimir los mejores hiperparámetros y la mejor precisión
print("Mejores hiperparámetros: ", study.best_params)
print("Mejor F1-score de validación: ", study.best_value)

# Entrenar el modelo final con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_clf = MultinomialNB(
    alpha=study.best_params['alpha'],
    fit_prior=study.best_params['fit_prior']
)
best_clf.fit(X_train, y_train)

[I 2025-09-07 22:10:26,735] A new study created in memory with name: no-name-bd455a9d-3ee8-40e8-865f-48e469fe3896
[I 2025-09-07 22:10:27,096] Trial 0 finished with value: 0.7541366075859119 and parameters: {'alpha': 0.002418860919523551, 'fit_prior': True}. Best is trial 0 with value: 0.7541366075859119.
[I 2025-09-07 22:10:27,455] Trial 1 finished with value: 0.741477957656946 and parameters: {'alpha': 0.0001811097245411849, 'fit_prior': True}. Best is trial 0 with value: 0.7541366075859119.
[I 2025-09-07 22:10:27,799] Trial 2 finished with value: 0.7537580797358405 and parameters: {'alpha': 0.0014542867544769158, 'fit_prior': False}. Best is trial 0 with value: 0.7541366075859119.
[I 2025-09-07 22:10:28,137] Trial 3 finished with value: 0.6684934091185349 and parameters: {'alpha': 0.38366894303052346, 'fit_prior': True}. Best is trial 0 with value: 0.7541366075859119.
[I 2025-09-07 22:10:28,473] Trial 4 finished with value: 0.7174906251260881 and parameters: {'alpha': 0.1451089882913

KeyboardInterrupt: 

In [ ]:
y_pred_optimizado = best_clf.predict(X_test)
f1_score(y_test, y_pred_optimizado, average='macro')

0.6894692817054662

In [ ]:
# Definir la función objetivo para Optuna
def objective_complementnb(trial):
    # Sugerir hiperparámetros a optimizar
    alpha = trial.suggest_float('alpha', 1e-4, 10.0, log=True)  # Parámetro de suavizado en escala logarítmica
    fit_prior = trial.suggest_categorical('fit_prior', [True, False])  # Si se aprenden las probabilidades previas de las clases
    
    # Inicializar el clasificador con los hiperparámetros sugeridos
    clf = ComplementNB(alpha=alpha, fit_prior=fit_prior)
    
    # Configurar validación cruzada estratificada con 5 pliegues
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    # Iterar sobre los pliegues de validación cruzada
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Dividir los datos en entrenamiento y validación
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        # Entrenar el modelo
        clf.fit(X_tr, y_tr)
        # Predecir y evaluar en el conjunto de validación
        y_pred = clf.predict(X_val)
        score = f1_score(y_val, y_pred, average='macro')
        scores.append(score)
    
    # Devolver la precisión media de los pliegues
    return np.mean(scores)

# Crear un estudio de Optuna para maximizar la precisión
study_complementnb = optuna.create_study(direction='maximize')
# Ejecutar la optimización con un máximo de 30 pruebas
study_complementnb.optimize(objective_complementnb, n_trials=30)

# Imprimir los mejores hiperparámetros y la mejor precisión
print("Mejores hiperparámetros: ", study_complementnb.best_params)
print("Mejor F1-score de validación: ", study_complementnb.best_value)

# Entrenar el modelo final con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_clf_complementnb = ComplementNB(
    alpha=study_complementnb.best_params['alpha'],
    fit_prior=study_complementnb.best_params['fit_prior']
)
best_clf_complementnb.fit(X_train, y_train)

[I 2025-09-07 11:16:36,294] A new study created in memory with name: no-name-ae2f6e0b-57ac-4251-adda-30a3e454eded
[I 2025-09-07 11:16:36,630] Trial 0 finished with value: 0.7193585060642885 and parameters: {'alpha': 0.0012858633855637605, 'fit_prior': False}. Best is trial 0 with value: 0.7193585060642885.
[I 2025-09-07 11:16:36,962] Trial 1 finished with value: 0.7601018835900443 and parameters: {'alpha': 0.3303165817702721, 'fit_prior': False}. Best is trial 1 with value: 0.7601018835900443.
[I 2025-09-07 11:16:37,288] Trial 2 finished with value: 0.7558925411703783 and parameters: {'alpha': 0.03785651242488598, 'fit_prior': True}. Best is trial 1 with value: 0.7601018835900443.
[I 2025-09-07 11:16:37,618] Trial 3 finished with value: 0.7379050140293708 and parameters: {'alpha': 0.008142932694501538, 'fit_prior': False}. Best is trial 1 with value: 0.7601018835900443.
[I 2025-09-07 11:16:37,948] Trial 4 finished with value: 0.7046351145273082 and parameters: {'alpha': 0.0002882844350

Mejores hiperparámetros:  {'alpha': 0.1962092602879557, 'fit_prior': False}
Mejor F1-score de validación:  0.7643122055499413


ComplementNB(alpha=0.1962092602879557, fit_prior=False)

In [ ]:
y_pred_complementnb = best_clf_complementnb.predict(X_test)
f1_score(y_test, y_pred_complementnb, average='macro')

0.6999155081250106

### Consigna 4
**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.